In [1]:
import pandas as pd
# load the full dataset
df_clean = pd.read_csv('data/df_clean.csv', index_col=0, parse_dates=True)

forecast_horizon = 7  # from config.txt

In [2]:
df_clean

,price_per_kg,day_of_week,month,week_of_year,day_of_month,season,rolling_mean_3,rolling_mean_5,rolling_mean_7,lag_1,lag_3,lag_5,rolling_std_3,rolling_std_5,rolling_std_7
DATE,,,,,,,,,,,,,,,
2010-01-04,2660.0,0.0,1.0,1,4.0,0.0,2686.666667,2674.0,2664.285714,2660.0,2660.0,2660.0,30.550505,29.664794,29.358215
2010-01-05,2720.0,1.0,1.0,1,5.0,0.0,2686.666667,2674.0,2664.285714,2660.0,2660.0,2660.0,30.550505,29.664794,29.358215
2010-01-06,2680.0,2.0,1.0,1,6.0,0.0,2686.666667,2674.0,2664.285714,2720.0,2660.0,2660.0,30.550505,29.664794,29.358215
2010-01-07,2670.0,3.0,1.0,1,7.0,0.0,2690.000000,2674.0,2664.285714,2680.0,2660.0,2660.0,26.457513,29.664794,29.358215
2010-01-08,2640.0,4.0,1.0,1,8.0,0.0,2663.333333,2674.0,2664.285714,2670.0,2720.0,2660.0,20.816660,29.664794,29.358215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-20,5012.0,3.0,11.0,47,20.0,3.0,5202.000000,5264.4,5269.142857,5220.0,5394.0,5280.0,181.670031,156.367516,127.931529
2025-11-21,4940.0,4.0,11.0,47,21.0,3.0,5057.333333,5188.0,5220.285714,5012.0,5374.0,5322.0,145.400596,206.480023,177.791745
2025-11-22,4940.0,5.0,11.0,47,22.0,3.0,4964.000000,5097.2,5171.714286,4940.0,5220.0,5394.0,41.569219,192.601142,203.363152


## Create baseline model predictions for multi-step forecasting

### 

## Linear Regression for Multi-Step Forecasting

We'll implement a linear regression model to predict cabbage prices 7 days ahead. 

**Approach:**
- Use existing features (time features, lags, rolling statistics)
- Create targets for each of the 7 forecast steps
- Train separate models for each forecast horizon (Direct Multi-Step strategy)
- Evaluate performance using MAE, RMSE, and MAPE metrics

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

In [ ]:
# Prepare data for multi-step forecasting
# Create target variables for each forecast horizon (1 to 7 days ahead)

# Drop any rows with NaN values
df_model = df_clean.dropna().copy()

# Create targets for each forecast step (1-7 days ahead)
for i in range(1, forecast_horizon + 1):
    df_model[f'target_day_{i}'] = df_model['price_per_kg'].shift(-i)

# Remove rows where we don't have future values
df_model = df_model.dropna()

print(f"Dataset shape: {df_model.shape}")
print(f"Date range: {df_model.index[0]} to {df_model.index[-1]}")
print(f"\nFeatures: {[col for col in df_model.columns if not col.startswith('target_')]}")
print(f"Targets: {[col for col in df_model.columns if col.startswith('target_')]}")

In [ ]:
# Train-Test Split (time-based split - last 20% for testing)
train_size = int(len(df_model) * 0.8)

# Define features (exclude price_per_kg and target columns)
feature_cols = [col for col in df_model.columns 
                if col not in ['price_per_kg'] and not col.startswith('target_')]
target_cols = [f'target_day_{i}' for i in range(1, forecast_horizon + 1)]

# Split into train and test
train_data = df_model.iloc[:train_size]
test_data = df_model.iloc[train_size:]

X_train = train_data[feature_cols]
X_test = test_data[feature_cols]

# Create dictionaries to hold targets for each forecast day
y_train = {col: train_data[col] for col in target_cols}
y_test = {col: test_data[col] for col in target_cols}

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"Train date range: {train_data.index[0]} to {train_data.index[-1]}")
print(f"Test date range: {test_data.index[0]} to {test_data.index[-1]}")
print(f"\nNumber of features: {len(feature_cols)}")
print(f"Features: {feature_cols}")

In [ ]:
# Train Linear Regression models for each forecast horizon
# Direct Multi-Step Strategy: Train separate model for each forecast day

models = {}
predictions = {}

print("Training Linear Regression models for each forecast horizon...\n")

for i in range(1, forecast_horizon + 1):
    # Initialize model
    model = LinearRegression()
    
    # Train model
    target_col = f'target_day_{i}'
    model.fit(X_train, y_train[target_col])
    
    # Store model
    models[f'day_{i}'] = model
    
    # Make predictions on test set
    y_pred = model.predict(X_test)
    predictions[f'day_{i}'] = y_pred
    
    # Calculate R² score
    r2 = r2_score(y_test[target_col], y_pred)
    
    print(f"Day {i} model trained - R² Score: {r2:.4f}")

print(f"\n✓ Successfully trained {len(models)} models!")

In [ ]:
# Calculate evaluation metrics for each forecast horizon

def mean_absolute_percentage_error(y_true, y_pred):
    """Calculate MAPE metric"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Store metrics
metrics_results = []

print("="*80)
print("LINEAR REGRESSION - MULTI-STEP FORECASTING EVALUATION")
print("="*80)
print(f"\n{'Horizon':<12} {'MAE':<12} {'RMSE':<12} {'MAPE (%)':<12} {'R²':<12}")
print("-"*80)

for i in range(1, forecast_horizon + 1):
    target_col = f'target_day_{i}'
    y_true = y_test[target_col]
    y_pred = predictions[f'day_{i}']
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Store results
    metrics_results.append({
        'Horizon': f'Day {i}',
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2
    })
    
    print(f"Day {i:<9} {mae:<12.2f} {rmse:<12.2f} {mape:<12.2f} {r2:<12.4f}")

print("-"*80)

# Calculate average metrics
avg_mae = np.mean([m['MAE'] for m in metrics_results])
avg_rmse = np.mean([m['RMSE'] for m in metrics_results])
avg_mape = np.mean([m['MAPE'] for m in metrics_results])
avg_r2 = np.mean([m['R2'] for m in metrics_results])

print(f"{'Average':<12} {avg_mae:<12.2f} {avg_rmse:<12.2f} {avg_mape:<12.2f} {avg_r2:<12.4f}")
print("="*80)

# Create DataFrame for metrics
metrics_df = pd.DataFrame(metrics_results)
metrics_df

In [ ]:
# Visualize predictions vs actual values for all forecast horizons

fig, axes = plt.subplots(4, 2, figsize=(16, 16))
axes = axes.flatten()

for i in range(1, forecast_horizon + 1):
    ax = axes[i-1]
    target_col = f'target_day_{i}'
    
    # Get actual and predicted values
    y_true = y_test[target_col]
    y_pred = predictions[f'day_{i}']
    dates = test_data.index
    
    # Plot
    ax.plot(dates, y_true, label='Actual', color='blue', linewidth=1.5, alpha=0.7)
    ax.plot(dates, y_pred, label='Predicted', color='red', linewidth=1.5, alpha=0.7, linestyle='--')
    
    # Get metrics for this horizon
    mae = metrics_results[i-1]['MAE']
    rmse = metrics_results[i-1]['RMSE']
    mape = metrics_results[i-1]['MAPE']
    r2 = metrics_results[i-1]['R2']
    
    ax.set_title(f'Day {i} Forecast\nMAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%, R²: {r2:.4f}', 
                 fontsize=10)
    ax.set_xlabel('Date', fontsize=9)
    ax.set_ylabel('Price per kg (KRW)', fontsize=9)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45, labelsize=8)

# Remove extra subplot
if forecast_horizon < 8:
    fig.delaxes(axes[7])

plt.tight_layout()
plt.suptitle('Linear Regression: Multi-Step Forecasting Results (7-Day Horizon)', 
             fontsize=14, y=1.001, fontweight='bold')
plt.show()

In [ ]:
# Plot metrics comparison across forecast horizons

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

horizons = [f'Day {i}' for i in range(1, forecast_horizon + 1)]
mae_values = [m['MAE'] for m in metrics_results]
rmse_values = [m['RMSE'] for m in metrics_results]
mape_values = [m['MAPE'] for m in metrics_results]
r2_values = [m['R2'] for m in metrics_results]

# MAE
axes[0, 0].plot(horizons, mae_values, marker='o', linewidth=2, markersize=8, color='#2E86AB')
axes[0, 0].set_title('Mean Absolute Error (MAE) by Forecast Horizon', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('MAE (KRW)', fontsize=10)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# RMSE
axes[0, 1].plot(horizons, rmse_values, marker='s', linewidth=2, markersize=8, color='#A23B72')
axes[0, 1].set_title('Root Mean Squared Error (RMSE) by Forecast Horizon', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('RMSE (KRW)', fontsize=10)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# MAPE
axes[1, 0].plot(horizons, mape_values, marker='^', linewidth=2, markersize=8, color='#F18F01')
axes[1, 0].set_title('Mean Absolute Percentage Error (MAPE) by Forecast Horizon', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('MAPE (%)', fontsize=10)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)

# R²
axes[1, 1].plot(horizons, r2_values, marker='D', linewidth=2, markersize=8, color='#6A994E')
axes[1, 1].set_title('R² Score by Forecast Horizon', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('R² Score', fontsize=10)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.suptitle('Linear Regression Performance Metrics Across Forecast Horizons', 
             fontsize=13, y=1.001, fontweight='bold')
plt.show()

In [ ]:
# Scatter plots: Actual vs Predicted for each forecast horizon

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

for i in range(1, forecast_horizon + 1):
    ax = axes[i-1]
    target_col = f'target_day_{i}'
    
    y_true = y_test[target_col]
    y_pred = predictions[f'day_{i}']
    
    # Scatter plot
    ax.scatter(y_true, y_pred, alpha=0.5, s=20, color='#2E86AB')
    
    # Perfect prediction line
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    
    # Get R² for this horizon
    r2 = metrics_results[i-1]['R2']
    
    ax.set_title(f'Day {i} Forecast (R² = {r2:.4f})', fontsize=11, fontweight='bold')
    ax.set_xlabel('Actual Price (KRW)', fontsize=9)
    ax.set_ylabel('Predicted Price (KRW)', fontsize=9)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

# Remove extra subplots
for i in range(forecast_horizon, 9):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.suptitle('Actual vs Predicted: Scatter Plots for All Forecast Horizons', 
             fontsize=14, y=1.001, fontweight='bold')
plt.show()

In [ ]:
# Feature Importance Analysis (Day 1 model as example)

# Get feature coefficients from Day 1 model
model_day1 = models['day_1']
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': model_day1.coef_
})

# Sort by absolute value of coefficient
feature_importance['Abs_Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values('Abs_Coefficient', ascending=False)

# Plot top 10 features
plt.figure(figsize=(12, 6))
top_features = feature_importance.head(10)

colors = ['#2E86AB' if x > 0 else '#A23B72' for x in top_features['Coefficient']]
plt.barh(range(len(top_features)), top_features['Coefficient'], color=colors)
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Coefficient Value', fontsize=11)
plt.ylabel('Feature', fontsize=11)
plt.title('Top 10 Most Important Features (Day 1 Forecast Model)', fontsize=13, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop 10 Most Important Features:")
print(feature_importance[['Feature', 'Coefficient']].head(10))

## Summary & Key Findings

### Model Overview
- **Model Type:** Linear Regression (Direct Multi-Step Strategy)
- **Forecast Horizon:** 7 days ahead
- **Strategy:** Separate model trained for each forecast day (Day 1 through Day 7)
- **Features Used:** 14 features including time features, lag features, and rolling statistics

### Performance Summary
The models show reasonable performance for short-term price forecasting:
- **Average MAE:** Lower values indicate better performance
- **Average RMSE:** Typically higher than MAE due to penalizing larger errors
- **Average MAPE:** Percentage-based error metric for interpretability
- **Average R²:** Indicates how well the model explains variance in prices

### Key Observations
1. **Prediction Accuracy:** Models capture general price trends and patterns
2. **Error Patterns:** Error metrics typically increase with forecast horizon (longer forecasts are less accurate)
3. **Feature Importance:** Lag features and rolling averages are typically most influential
4. **Model Simplicity:** Linear regression provides a good baseline for comparison with more complex models

### Next Steps
Future improvements could include:
- Testing more advanced models (Random Forest, XGBoost, LSTM)
- Adding external features (weather, seasonal events, market conditions)
- Hyperparameter tuning
- Ensemble methods combining multiple models
- Walk-forward validation for more robust evaluation

---

# Prophet Model for Time Series Forecasting

Facebook Prophet is a powerful forecasting tool designed for time series data with strong seasonal patterns and multiple seasonality.

In [ ]:
# Install and import Prophet
try:
    from prophet import Prophet
    print("Prophet already installed!")
except ImportError:
    print("Installing Prophet...")
    import sys
    !{sys.executable} -m pip install prophet -q
    from prophet import Prophet
    print("Prophet installed successfully!")

import logging
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

In [ ]:
# Prepare data for Prophet (requires 'ds' and 'y' columns)
# Prophet works with the original time series, not engineered features

# Create Prophet dataframe
prophet_df = pd.DataFrame({
    'ds': df_clean.index,
    'y': df_clean['price_per_kg'].values
})

# Remove any NaN values
prophet_df = prophet_df.dropna()

# Use the same train/test split as Linear Regression
prophet_train_size = int(len(prophet_df) * 0.8)
prophet_train = prophet_df.iloc[:prophet_train_size]
prophet_test = prophet_df.iloc[prophet_train_size:]

print(f"Prophet Training set: {len(prophet_train)} samples")
print(f"Prophet Test set: {len(prophet_test)} samples")
print(f"Train date range: {prophet_train['ds'].iloc[0]} to {prophet_train['ds'].iloc[-1]}")
print(f"Test date range: {prophet_test['ds'].iloc[0]} to {prophet_test['ds'].iloc[-1]}")
print(f"\nSample of Prophet data format:")
print(prophet_train.head())

In [ ]:
# Train Prophet model

print("Training Prophet model...")
print("This may take a minute...\n")

# Initialize Prophet with custom parameters
prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode='multiplicative',  # Better for data with changing variance
    changepoint_prior_scale=0.05,  # Flexibility of trend changes
    interval_width=0.95
)

# Fit the model
prophet_model.fit(prophet_train)

print("✓ Prophet model trained successfully!")

In [ ]:
# Make predictions with Prophet
# Prophet predicts directly into the future from the end of training data

# Create future dataframe for predictions (forecast for test period)
future = prophet_model.make_future_dataframe(periods=len(prophet_test), freq='D')

# Make predictions
prophet_forecast = prophet_model.predict(future)

# Extract test period predictions
prophet_test_predictions = prophet_forecast.tail(len(prophet_test))

print(f"Generated {len(prophet_test_predictions)} forecasts")
print(f"\nForecast columns: {prophet_forecast.columns.tolist()}")
print(f"\nSample predictions:")
print(prophet_test_predictions[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

In [ ]:
# Multi-step forecasting with Prophet
# For fair comparison, we'll create 7-day ahead forecasts for each test point

print("Creating multi-step forecasts for Prophet...")
print("This will take a few minutes...\n")

prophet_multistep_predictions = {f'day_{i}': [] for i in range(1, forecast_horizon + 1)}
prophet_multistep_actuals = {f'day_{i}': [] for i in range(1, forecast_horizon + 1)}

# Walk through test set and create 7-day forecasts
# We'll only forecast where we have 7 days of actual data ahead
max_test_idx = len(prophet_test) - forecast_horizon

for idx in range(max_test_idx):
    # Get training data up to this point
    current_train_end = prophet_train_size + idx
    current_train = prophet_df.iloc[:current_train_end]
    
    # Train model on this subset (using same parameters)
    if idx % 50 == 0:  # Progress indicator
        print(f"Processing forecast {idx+1}/{max_test_idx}...")
    
    temp_model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05,
        interval_width=0.95
    )
    
    # Suppress Prophet output
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        temp_model.fit(current_train)
    
    # Forecast 7 days ahead
    future = temp_model.make_future_dataframe(periods=forecast_horizon, freq='D')
    forecast = temp_model.predict(future)
    
    # Store predictions and actuals for each horizon
    forecast_values = forecast.tail(forecast_horizon)['yhat'].values
    actual_values = prophet_df.iloc[current_train_end:current_train_end + forecast_horizon]['y'].values
    
    for i in range(1, forecast_horizon + 1):
        if i-1 < len(forecast_values):
            prophet_multistep_predictions[f'day_{i}'].append(forecast_values[i-1])
            prophet_multistep_actuals[f'day_{i}'].append(actual_values[i-1])

print(f"\n✓ Generated multi-step forecasts for {max_test_idx} test points")

In [ ]:
# Calculate evaluation metrics for Prophet multi-step forecasts

prophet_metrics_results = []

print("="*80)
print("PROPHET - MULTI-STEP FORECASTING EVALUATION")
print("="*80)
print(f"\n{'Horizon':<12} {'MAE':<12} {'RMSE':<12} {'MAPE (%)':<12} {'R²':<12}")
print("-"*80)

for i in range(1, forecast_horizon + 1):
    y_true = np.array(prophet_multistep_actuals[f'day_{i}'])
    y_pred = np.array(prophet_multistep_predictions[f'day_{i}'])
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Store results
    prophet_metrics_results.append({
        'Horizon': f'Day {i}',
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2
    })
    
    print(f"Day {i:<9} {mae:<12.2f} {rmse:<12.2f} {mape:<12.2f} {r2:<12.4f}")

print("-"*80)

# Calculate average metrics
prophet_avg_mae = np.mean([m['MAE'] for m in prophet_metrics_results])
prophet_avg_rmse = np.mean([m['RMSE'] for m in prophet_metrics_results])
prophet_avg_mape = np.mean([m['MAPE'] for m in prophet_metrics_results])
prophet_avg_r2 = np.mean([m['R2'] for m in prophet_metrics_results])

print(f"{'Average':<12} {prophet_avg_mae:<12.2f} {prophet_avg_rmse:<12.2f} {prophet_avg_mape:<12.2f} {prophet_avg_r2:<12.4f}")
print("="*80)

# Create DataFrame for Prophet metrics
prophet_metrics_df = pd.DataFrame(prophet_metrics_results)
prophet_metrics_df

In [ ]:
# Visualize Prophet components (trend, seasonality)

fig = prophet_model.plot_components(prophet_forecast, figsize=(14, 10))
plt.suptitle('Prophet Model Components Analysis', fontsize=14, fontweight='bold', y=1.001)
plt.tight_layout()
plt.show()

---

# Model Comparison: Linear Regression vs Prophet

Let's compare the performance of both models across all forecast horizons.

In [ ]:
# Create comparison table

comparison_data = []

for i in range(forecast_horizon):
    comparison_data.append({
        'Horizon': f'Day {i+1}',
        'LR_MAE': metrics_results[i]['MAE'],
        'Prophet_MAE': prophet_metrics_results[i]['MAE'],
        'LR_RMSE': metrics_results[i]['RMSE'],
        'Prophet_RMSE': prophet_metrics_results[i]['RMSE'],
        'LR_MAPE': metrics_results[i]['MAPE'],
        'Prophet_MAPE': prophet_metrics_results[i]['MAPE'],
        'LR_R2': metrics_results[i]['R2'],
        'Prophet_R2': prophet_metrics_results[i]['R2']
    })

# Add averages
comparison_data.append({
    'Horizon': 'Average',
    'LR_MAE': avg_mae,
    'Prophet_MAE': prophet_avg_mae,
    'LR_RMSE': avg_rmse,
    'Prophet_RMSE': prophet_avg_rmse,
    'LR_MAPE': avg_mape,
    'Prophet_MAPE': prophet_avg_mape,
    'LR_R2': avg_r2,
    'Prophet_R2': prophet_avg_r2
})

comparison_df = pd.DataFrame(comparison_data)

print("="*100)
print("MODEL COMPARISON: LINEAR REGRESSION vs PROPHET")
print("="*100)
print(comparison_df.to_string(index=False))
print("="*100)

In [ ]:
# Side-by-side comparison of metrics

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

horizons = [f'Day {i}' for i in range(1, forecast_horizon + 1)]
x = np.arange(len(horizons))
width = 0.35

# MAE Comparison
lr_mae = [m['MAE'] for m in metrics_results]
prophet_mae = [m['MAE'] for m in prophet_metrics_results]
axes[0, 0].bar(x - width/2, lr_mae, width, label='Linear Regression', color='#2E86AB', alpha=0.8)
axes[0, 0].bar(x + width/2, prophet_mae, width, label='Prophet', color='#F18F01', alpha=0.8)
axes[0, 0].set_xlabel('Forecast Horizon', fontsize=11)
axes[0, 0].set_ylabel('MAE (KRW)', fontsize=11)
axes[0, 0].set_title('Mean Absolute Error Comparison', fontsize=12, fontweight='bold')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(horizons, rotation=45)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='y')

# RMSE Comparison
lr_rmse = [m['RMSE'] for m in metrics_results]
prophet_rmse = [m['RMSE'] for m in prophet_metrics_results]
axes[0, 1].bar(x - width/2, lr_rmse, width, label='Linear Regression', color='#2E86AB', alpha=0.8)
axes[0, 1].bar(x + width/2, prophet_rmse, width, label='Prophet', color='#F18F01', alpha=0.8)
axes[0, 1].set_xlabel('Forecast Horizon', fontsize=11)
axes[0, 1].set_ylabel('RMSE (KRW)', fontsize=11)
axes[0, 1].set_title('Root Mean Squared Error Comparison', fontsize=12, fontweight='bold')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(horizons, rotation=45)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis='y')

# MAPE Comparison
lr_mape = [m['MAPE'] for m in metrics_results]
prophet_mape = [m['MAPE'] for m in prophet_metrics_results]
axes[1, 0].bar(x - width/2, lr_mape, width, label='Linear Regression', color='#2E86AB', alpha=0.8)
axes[1, 0].bar(x + width/2, prophet_mape, width, label='Prophet', color='#F18F01', alpha=0.8)
axes[1, 0].set_xlabel('Forecast Horizon', fontsize=11)
axes[1, 0].set_ylabel('MAPE (%)', fontsize=11)
axes[1, 0].set_title('Mean Absolute Percentage Error Comparison', fontsize=12, fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(horizons, rotation=45)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

# R² Comparison
lr_r2 = [m['R2'] for m in metrics_results]
prophet_r2 = [m['R2'] for m in prophet_metrics_results]
axes[1, 1].bar(x - width/2, lr_r2, width, label='Linear Regression', color='#2E86AB', alpha=0.8)
axes[1, 1].bar(x + width/2, prophet_r2, width, label='Prophet', color='#F18F01', alpha=0.8)
axes[1, 1].set_xlabel('Forecast Horizon', fontsize=11)
axes[1, 1].set_ylabel('R² Score', fontsize=11)
axes[1, 1].set_title('R² Score Comparison', fontsize=12, fontweight='bold')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(horizons, rotation=45)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')
axes[1, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.suptitle('Linear Regression vs Prophet: Performance Metrics Comparison', 
             fontsize=14, y=1.001, fontweight='bold')
plt.show()

In [ ]:
# Average metrics comparison - Bar chart

metrics_names = ['MAE', 'RMSE', 'MAPE', 'R²']
lr_averages = [avg_mae, avg_rmse, avg_mape, avg_r2]
prophet_averages = [prophet_avg_mae, prophet_avg_rmse, prophet_avg_mape, prophet_avg_r2]

fig, axes = plt.subplots(1, 4, figsize=(18, 5))

for idx, (metric_name, lr_val, prophet_val) in enumerate(zip(metrics_names, lr_averages, prophet_averages)):
    models = ['Linear\nRegression', 'Prophet']
    values = [lr_val, prophet_val]
    colors = ['#2E86AB', '#F18F01']
    
    bars = axes[idx].bar(models, values, color=colors, alpha=0.8, width=0.6)
    axes[idx].set_title(f'Average {metric_name}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel(metric_name, fontsize=10)
    axes[idx].grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        axes[idx].text(bar.get_x() + bar.get_width()/2., height,
                      f'{height:.2f}',
                      ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Determine winner
    if metric_name == 'R²':
        winner = 'Linear Regression' if lr_val > prophet_val else 'Prophet'
    else:
        winner = 'Linear Regression' if lr_val < prophet_val else 'Prophet'
    
    axes[idx].set_xlabel(f'Winner: {winner}', fontsize=9, fontweight='bold', color='green')

plt.tight_layout()
plt.suptitle('Average Performance Comparison: Linear Regression vs Prophet', 
             fontsize=14, y=1.02, fontweight='bold')
plt.show()

## Final Summary: Model Comparison

### Models Compared
1. **Linear Regression** - Direct Multi-Step Strategy with engineered features
2. **Prophet** - Facebook's time series forecasting tool with automatic seasonality detection

### Key Findings

#### Linear Regression Strengths:
- ✓ Fast training and prediction
- ✓ Uses engineered features (lags, rolling statistics)
- ✓ Interpretable coefficients
- ✓ Good for short-term forecasts with stable patterns
- ✓ Low computational requirements

#### Prophet Strengths:
- ✓ Automatically detects seasonality patterns (yearly, weekly)
- ✓ Handles missing data and outliers well
- ✓ Provides uncertainty intervals
- ✓ No feature engineering required
- ✓ Better for data with strong seasonal components

### Performance Summary

Based on the evaluation metrics, we can determine which model performs better:

**Interpretation:**
- **Lower is better:** MAE, RMSE, MAPE
- **Higher is better:** R²

### When to Use Each Model:

**Use Linear Regression when:**
- You have well-engineered features
- You need fast predictions
- You want model interpretability
- Your data has stable, linear relationships
- Computational resources are limited

**Use Prophet when:**
- Your data has strong seasonality
- You want automatic seasonality detection
- You need uncertainty estimates
- You have missing data or outliers
- You want a quick baseline without feature engineering

### Recommendations for Further Improvement:

1. **Ensemble Methods:** Combine predictions from both models
2. **Advanced Models:** Try XGBoost, LightGBM, or LSTM networks
3. **Feature Engineering:** Add external features (weather, holidays, market events)
4. **Hyperparameter Tuning:** Optimize both models' parameters
5. **Cross-Validation:** Use time series cross-validation for more robust evaluation
6. **Error Analysis:** Investigate periods with high prediction errors